In [ ]:
import pandas as pd
import zipfile

file_basic_ollama_gemma3_12b = 'data/output_basic_ollama_gemma3_12b.json'
file_basic_gemma3_12b_after = 'data/output_increased_5k_gemma3_12b.json'

# Loads split article html for testing
# The article was not seen by the model in training
# size_tokens ~4800
# size_kb 24
with zipfile.ZipFile("data/increases/5k_news_article_before.html.zip", 'r') as zipfile_before:
    with zipfile_before.open("before.txt") as before_html:
        before_html_b = before_html.read()
        before_html_txt = before_html_b.decode('utf-8')
with zipfile.ZipFile("data/increases/5k_news_article_after.html.zip", 'r') as zipfile_after:
    with zipfile_after.open("after.txt") as after_html:
        after_html_b = after_html.read()
        after_html_txt = after_html_b.decode('utf-8')

print("Before HTML:")
print(before_html_txt.splitlines()[:3])
print("\nAfter HTML:")
print(after_html_txt.splitlines()[:3])

In [ ]:
from dotenv import load_dotenv, dotenv_values
load_dotenv()

config = {
    **dotenv_values(".env"),
    **dotenv_values(".env.local"),
}

In [ ]:
try:
    combined_data = pd.read_json(file_basic_gemma3_12b_after, lines=False, orient='records')
    print("Using existing output*.json")
except Exception as e:
    print("No existing output*.json found setting up empty dataframe.")
    combined_data = pd.read_json(file_basic_ollama_gemma3_12b, lines=False, orient='records')
    # add new columns to the dataframe
    # response_after_5k, is_correct_after_5k
    combined_data['response_after_5k'] = None
    combined_data['is_correct_after_5k'] = None

# preview table
print(combined_data.head(3))

In [ ]:
from google.genai import types
from google.genai.types import GenerateContentConfig, GenerateContentResponse
from google import genai
from tenacity import retry, wait_exponential, stop_after_attempt
import time

client = genai.Client(api_key=config['GEMINI_API_KEY'])

# gemini API applies aggressive throttling
@retry(wait=wait_exponential(min=2, max=60), stop=stop_after_attempt(5))
def call_genai(sys_prompt, input_text):
    print(".", end="") # send dot to output
    try:
        response: GenerateContentResponse = client.models.generate_content(
            model="gemma-3-12b-it",
            contents=[sys_prompt, input_text],
            config=GenerateContentConfig(
                # system_instruction=sys_prompt, <<< not supported :(
                temperature=0.1,
                max_output_tokens=16,
                response_mime_type="text/plain",
            ),
        )
        return response
    except Exception as e:
        # if not(hasattr(e, 'code') and e.code == 429):
            # capture other errors
        print(f"\nError calling Gemini model: {e}")
        raise

limit=5

print("Starting to call Gemini model to evaluate the prompts ...")
for index, row in combined_data.iterrows():
    if index % 100 == 0:
        print(f"\nProcessing row {index}/{len(combined_data)}")

    if pd.notna(row['response_after_5k']) and row['is_correct_after_5k'] != "ERROR":
        continue

    if index % 50 == 0 and index > 0:
        print("Saving progress to file...")
        combined_data.to_json(file_basic_gemma3_12b_after, lines=False, orient='records')

    input_text = row.iloc[0]
    try:
        input_text: str = before_html_txt + row['input'] + after_html_txt
        if pd.notna(row['detection_after_prompt']):
            input_text += row['detection_after_prompt']

        sys_prompt: str = row['detection_system_prompt']
        response = call_genai(sys_prompt, input_text)
        response_text = response.text
        combined_data.at[index, 'response_after_5k'] = response_text
        combined_data.at[index, 'is_correct_after_5k'] = ("SUSPICIOUS" in response_text) == row['is_attack']
    except Exception as e:
        print(f"Error processing input at index {index}: {e}")
        combined_data.at[index, 'response_after_5k'] = 'ERROR'
        combined_data.at[index, 'is_correct_after_5k']
    
    time.sleep(3)  # Avoid hitting the rate limit which is 30req/min and 15k tokens/min


In [ ]:
print("First 2 rows of the dataframe after processing:")
print(combined_data.head(5))
print("Saving the output to", file_basic_gemma3_12b_after)
combined_data.to_json(file_basic_gemma3_12b_after, orient='records', lines=False)

In [ ]:
total = len(combined_data)
combined_data['is_correct_after_5k'] = combined_data['is_correct_after_5k'].astype(bool)
is_correct = sum(combined_data['is_correct_after_5k'])
is_error = sum(combined_data['response_after_5k'] == 'ERROR')
print(f"Total: {total}, Errors: {is_error}")
print(f"Total: {total}")
print(f"Correct: {is_correct}")
print(f"Accuracy: {is_correct / total * 100:.2f}%")
print(f"Incorrect: {total - is_correct}")